# Parallel teleport

Start by initializing the environment

In [ ]:
from qsharp import init, TargetProfile, code, compile, circuit
from qsharp_widgets import Atoms, Circuit, Histogram
from qsharp.passes import transform, trace, AC1K

init(target_profile=TargetProfile.Base)

## Teleportation

This uses controlled operations rather than classical measurement to avoid mid-circuit measurement.

It is written to spread multiple instances of teleportation across the available qubits

In [ ]:
%%qsharp

operation Teleport(message : Qubit, target : Qubit, aux: Qubit) : Result[] {
    // Create some entanglement that we can use to send our message.
    H(aux);
    CNOT(aux, target);

    // Encode the message into the entangled pair.
    CNOT(message, aux);
    H(message);

    CNOT(aux, target);
    Controlled Z([message], target);

    MResetEachZ([message, target, aux])
}

operation Run(instances: Int) : Result[] {
    // Partitions the teleport instances across rows and columns on the machine
    let cols = if instances >= 12 {36} else {instances * 3};
    let rows = (instances + 11) / 12;  // 1 to 12 = 1, 13 to 24 = 2, etc.

    use qubits = Qubit[instances * 3];
    mutable results: Result[] = [];

    for i in 0..instances-1 {
        let rowId = i / 12;
        let colId = (i % 12) * 3;
        let idx = colId + (rowId * 36);
        let result = Teleport(qubits[idx], qubits[idx + 1], qubits[idx + 2]);
        results += result;
    }

    return results;
}


## Circuit

Verify the circuit for two rounds of teleportation

In [ ]:
Circuit(circuit(code.Run, 2))


## QIR

Show the generated QIR for one round of teleportation

In [ ]:
%time qir = compile(code.Run, 1)
len(str(qir).splitlines())
print(qir)

# Simulation

Transform the QIR to the AC1000 requirements and visulize

In [ ]:
transformed_qir = transform(qir, verbose=True)
len(str(transformed_qir).splitlines())
# print(transformed_qir)

In [ ]:
Atoms(
    machine_layout = AC1K,
    trace_data = trace(transformed_qir),
)

## Scale

Now run 36 rounds of teleportation in parallel and visualize

In [ ]:
qir120 = compile(code.Run, 120)
transformed_qir120 = transform(qir120, verbose=True)
Atoms(machine_layout = AC1K, trace_data = trace(transformed_qir120))

## Clifford simulation

Run noiseless for 10,000 shots

In [ ]:
from qsharp._simulation import run_qir, NoiseConfig
from qsharp.passes import transform_to_clifford

# Ensure it's Clifford only
clifford_qir = transform_to_clifford(qir120)

# Run the simulation
output = run_qir(str(clifford_qir), 10, NoiseConfig())

# Show the results
Histogram(output)

In [ ]:
# TODO: Run with noise to show some loss

In [ ]:
# How long does 10,000 runs take?
%time output = run_qir(str(clifford_qir), 10000, NoiseConfig())